#### Getting Started 
We'll first get our basic dependencies set up and ready to go. Since we want to use real, noisy devices for our calibration experiments in this notebook, we need to load our IBMQ account and set the appropriate backend.

In [19]:
from qiskit.tools.jupyter import *

In [21]:
from qiskit import IBMQ

#IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

We verify that the backend supports Pulse features by checking the backend configuration. The config provides us with general information about the structure of the backend setup.

In [22]:
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

#### Building Pulse Schedules

Pulse programs, which are called `Schedule`s, describe instruction sequences for the control electronics. We build `Schedule`s using the Pulse Builder. It's easy to initialize a schedule:

In [1]:
from qiskit import pulse

with pulse.build(name="my_example") as my_program:
    # Add instructions here
    pass
my_program

ScheduleBlock(, name="my_example", transform=AlignLeft())

You can see that there are no instructions yet. The next section of this page will explain each of the instructions you might add to a schedule, and the last section will describe various _alignment contexts_, which determine how instructions are placed in time relative to one another.

#### `Schedule` Instructions

 - [`delay(duration, channel)`](#delay)
 - [`play(pulse, channel)`](#play)
 - [`set_frequency(frequency, channel)`](#frequency)
 - [`shift_phase(phase, channel)`](#phase)
 - [`acquire(duration, channel, mem_slot, reg_slot)`](#acquire)

Each instruction type has its own set of operands. As you can see above, they each include at least one `Channel` to specify where the instruction will be applied.

**Channels** are labels for signal lines from the control hardware to the quantum chip.

 - `DriveChannel`s are typically used for _driving_ single qubit rotations,
 - `ControlChannel`s are typically used for multi-qubit gates or additional drive lines for tunable qubits, 
 - `MeasureChannel`s are specific to transmitting pulses which stimulate readout, and
 - `AcquireChannel`s are used to trigger digitizers which collect readout signals.
 
`DriveChannel`s, `ControlChannel`s, and `MeasureChannel`s are all `PulseChannel`s; this means that they support _transmitting_ pulses, whereas the `AcquireChannel` is a receive channel only and cannot play waveforms.

For the following examples, we will create one `DriveChannel` instance for each `Instruction` that accepts a `PulseChannel`. Channels take one integer `index` argument. Except for `ControlChannel`s, the index maps trivially to the qubit label.

In [24]:
from qiskit.pulse import DriveChannel

channel = DriveChannel(0)

The pulse `Schedule` is independent of the backend it runs on. However, we can build our program in a context that is aware of the target backend by supplying it to `pulse.build`. When possible you should supply a backend. By using the channel accessors `pulse.<type>_channel(<idx>)` we can make sure we are only using available device resources.

In [29]:
from qiskit.test.mock import FakeValencia

backend = FakeValencia()


with pulse.build(backend=backend, name="backend_aware") as backend_aware_program:
    channel = pulse.drive_channel(5)
    #print(pulse.num_qubits())

BackendConfigurationError: 'Invalid index for 5-qubit system.'